In [120]:
import pandas as pd
import numpy as np
import os, sys, re

from _config import SEQUENCING_READS_META, REDUCED_LIB

sequencing_info = pd.DataFrame()
for nm,vals in sequencing_reads_meta.items():
    #read sequencing maps and experimental design files
    reads_place = vals["reads_place"]
    lib_file = vals["lib_file"]
    reads_files = os.listdir(reads_place)
    #lib = pd.read_csv(lib_file,names=["Code"])
    info = pd.read_csv(lib_file) 

    #add metadata to sequencing info DF
    info = pd.concat([info,
                      info.apply(
        lambda x:pd.Series( re.compile("(?P<Name>D[^\s]+) : (?P<Description>[^()]*)\((?P<Index>[^)]*)\)")
                           .search(x["code"]).groupdict()),axis=1)]
                     ,axis=1)
    info["nm"] = nm
    info["reads_place"] = reads_place
    reads_files =os.listdir(reads_place)
    
    info["reads_location"] = info.Name.apply(
        lambda x: [e for e in reads_files if x in e]).apply(
            lambda x:os.path.join(reads_place,x[0]) if len(x)>0 
        else None)
    
    #find fastq files using sequencing info DF
    if False in set(pd.notna(info.reads_location)):
        raise Exception(f"missing reads data file for {nm}")
    info["fastq"] = info.apply(lambda x:[fn for fn in os.listdir(x.reads_location) if ~(pd.isna(x.reads_location)) & ( fn[-5:]=="fastq")][0],axis=1)
    info["fastq_path"] = info["reads_location"] +"/"+ info["fastq"]
    sequencing_info = sequencing_info.append(info)
     
 

In [139]:
def rc(s):
    return "".join([{"N":"N","A":"T","G":"C","T":"A","C":"G"}[l] for l in s][::-1])

In [111]:
from Bio import pairwise2

In [119]:
seqs = []
for fpath in sequencing_info["fastq_path"].iloc[::10]:
    num_bad_q = 0
    num_total = 0
    
    with open(fpath) as f:
        for i,l in enumerate(f):
            if i%4 == 0:
                h1 = l.strip()
                continue
            if i%4 == 1:
                r1 = l.strip()
                continue
            if i%4 == 3:
                num_total+=1
                qs = l.strip()
                quals = [ord(s)-33 for s in qs]
                if np.mean(quals) < 32:        
                    num_bad_q += 1
                    continue
                #print( r1)
                #print(np.mean(quals))
                #print(REDUCED_LIB_DESIGN["Designed gRNA (NGG orientation, 19 or 20)"].apply(lambda x : x[:10] in rc(r1)).nonzero())
                seqs.append(r1)
                if i > 10000: break

    print(f"{num_bad_q} bad reads of {num_total} total")


/cluster/bh0085/shortreads/190328Gif/D19-170003-3778L/190328Gif_D19-170003_NA_sequence.fastq
660 bad reads of 2502 total
/cluster/bh0085/shortreads/190328Gif/D19-170013-3778L/190328Gif_D19-170013_NA_sequence.fastq
710 bad reads of 2501 total
/cluster/bh0085/shortreads/190328Gif/D19-170023-3778L/190328Gif_D19-170023_NA_sequence.fastq
412 bad reads of 2501 total
/cluster/bh0085/shortreads/190328Gif/D19-170033-3778L/190328Gif_D19-170033_NA_sequence.fastq
625 bad reads of 2501 total
/cluster/bh0085/shortreads/190329Gif/D19-171012-3779L/190329Gif_D19-171012_NA_sequence.fastq
628 bad reads of 2501 total
/cluster/bh0085/shortreads/190329Gif/D19-171022-3779L/190329Gif_D19-171022_NA_sequence.fastq
591 bad reads of 2501 total
/cluster/bh0085/shortreads/190502Gif/D19-194008-3846G/190502Gif_D19-194008_2_sequence.fastq
2776 bad reads of 2800 total
/cluster/bh0085/shortreads/190502Gif/D19-194018-3846G/190502Gif_D19-194018_2_sequence.fastq
2509 bad reads of 2541 total
/cluster/bh0085/shortreads/19050

In [148]:
len("TGTAGATGGAAAGGACGAAACACCGCTCATCTCGGAACCACGTGAGTTTAAGAGCTATGCTGGAAACAGCATAGCA")

76